In [1]:
#-*- encoding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os.path
import sys
import numpy as np
import tensorflow as tf

import input_data
import models

/root/miniconda3/envs/tf1.13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/miniconda3/envs/tf1.13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/miniconda3/envs/tf1.13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/miniconda3/envs/tf1.13/lib/python3.7/site-packages/tensorf

In [2]:
def data_stats(train_data, val_data, test_data):
    """mean and std_dev

        Args:
            train_data: (36923, 490)
            val_data: (4445, 490)
            test_data: (4890, 490)

        Return: (mean, std_dev)

        Result:
            mean: -3.975149608704592, 220.81257374779565
            std_dev: 0.8934739293234528
    """
    print(train_data.shape, val_data.shape, test_data.shape)
    all_data = np.concatenate((train_data, val_data, test_data), axis=0)
    std_dev = 255. / (all_data.max() - all_data.min())
    # mean_ = all_data.mean()
    mean_ = 255. * all_data.min() / (all_data.min() - all_data.max())
    return (mean_, std_dev)

def fp32_to_uint8(r):
    # method 1
    # s = (r.max() - r.min()) / 255.
    # z = 255. - r.max() / s
    # q = r / s + z

    # method 2
    std_dev = 0.8934739293234528
    mean_ = 220.81257374779565
    q = r / std_dev + mean_
    q = q.astype(np.uint8)
    return q

def calc(interpreter, input_data, label):

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # print(input_details)
    # print(output_details)

    # Test model on random input data.
    # input_shape = input_details[0]['shape']
    # input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    # print(output_data)
    # print(label)

    return output_data

In [3]:
wanted_words = 'yes,no,up,down,left,right,on,off,stop,go'
sample_rate = 16000
clip_duration_ms = 1000
model_architecture = 'mobilenet-v3'
dct_coefficient_count = 10
batch_size = 1
window_size_ms = 40
window_stride_ms = 20
model_size_info = [4, 16, 10, 4, 2, 2, 16, 3, 3, 1, 1, 2, 32, 3, 3, 1, 1, 2, 32, 5, 5, 1, 1, 2]
data_url = 'http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
data_dir = '/tmp/speech_dataset/'
silence_percentage = 10.0
unknown_percentage = 10.0
testing_percentage = 10
validation_percentage = 10

In [4]:
tf.logging.set_verbosity(tf.logging.INFO)
sess = tf.InteractiveSession()
words_list = input_data.prepare_words_list(wanted_words.split(','))
model_settings = models.prepare_model_settings(
    len(words_list), sample_rate, clip_duration_ms, window_size_ms,
    window_stride_ms, dct_coefficient_count)

audio_processor = input_data.AudioProcessor(
    data_url, data_dir, silence_percentage,
    unknown_percentage,
    wanted_words.split(','), validation_percentage,
    testing_percentage, model_settings)

label_count = model_settings['label_count']
fingerprint_size = model_settings['fingerprint_size']

fingerprint_input = tf.placeholder(
    tf.float32, [None, fingerprint_size], name='fingerprint_input')

logits = models.create_model(
    fingerprint_input,
    model_settings,
    model_architecture,
    model_size_info,
    is_training=False)

ground_truth_input = tf.placeholder(
    tf.float32, [None, label_count], name='groundtruth_input')

predicted_indices = tf.argmax(logits, 1)
expected_indices = tf.argmax(ground_truth_input, 1)
correct_prediction = tf.equal(predicted_indices, expected_indices)
confusion_matrix = tf.confusion_matrix(
    expected_indices, predicted_indices, num_classes=label_count)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



Total number of layers: %d
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [178]:
tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/stem_conv.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_1_expansion.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_1_depthwise.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_1_projection.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_2_expansion.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_2_depthwise.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_2_protection.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_3_expansion.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_3_depthwise.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_3_projection.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/inverted_residual_3_add.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/AvgPool.lite'
# tflite_path = 'test_log/mobilenetv3_quant_eval/layers_lite_model/Conv2D.lite'

# Boss
# tflite_path = 'test_log/mobilenetv3_quant_eval/uint8input_8bit_calc_mean220_std0.89.lite'

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_path)
# interpreter = tf.lite.Interpreter(model_path="tflite_factory/swiftnet-uint8.lite")
interpreter.allocate_tensors()


In [181]:
def manual_int(input_data):
    ################## stem conv ##################
    print('stem conv')
    new_data = input_data.reshape(-1, 49, 10, 1)
    new_data = tf.convert_to_tensor(new_data, tf.float32, name='input')
    new_data =  new_data - 221.
    s_iw = 1.1192268133163452 * 0.0009845112217590213

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_stem_conv_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_stem_conv_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32, name='weight')
    weight = weight - 132.
    weight = tf.transpose(weight, perm=[1,2,0,3])

    bias = tf.convert_to_tensor(bias, tf.float32, name='bias')
    bias = 0.0011018913937732577 * bias
    print(weight)
    print(bias)

    output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,2,2,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None,  # 数据格式，与步长参数配合，决定移动方式
                name='stem_conv') # 名字，用于tensorboard图形显示时使用
    output_fp = s_iw * output

    output_add = tf.add(output_fp, bias, name='add')
    # output_uint8 = tf.cast(output, dtype=tf.uint8)
    output_add = output_add / 0.16148914396762848
    
    output_relu = tf.nn.relu(output_add)
    output_uint8 = tf.math.round(output_relu, name='round')
    output_uint8 = tf.cast(output_uint8, tf.uint8, name='uint8')
    add_2 = tf.identity(output_uint8)   # 给之后的做加法
    print()

    ################## inverted residual 1 expansion ##################
    print('inverted residual 1 expansion')
    new_data = tf.cast(output_uint8, tf.float32)
    s_iw = 0.16148914396762848 * 0.01888326182961464

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_expansion_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_expansion_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 146
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.003049441846087575
    print(bias)

    output = tf.nn.conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.27361148595809937
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    print()

    ################## inverted residual 1 depthwise ##################
    print('inverted residual 1 depthwise')
    new_data = tf.cast(output_uint8, tf.float32)
    s_iw = 0.27361148595809937 * 0.016701024025678635

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_depthwise_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_depthwise_depthwise_conv_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 127
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.0045695919543504715
    print(bias)

    output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.12676289677619934
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    print()

    ################## inverted residual 1 projection ##################
    print('inverted residual 1 projection')
    new_data = tf.cast(output_uint8, tf.float32)
    s_iw = 0.12676289677619934 * 0.007413254585117102

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_projection_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_1_projection_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 101
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.0009397256653755903
    print(bias)

    output = tf.nn.conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.16901935636997223 + 133
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    add_1 = tf.identity(output_uint8)
    print()

    ################## inverted residual 1 add ##################
    add_1 = tf.cast(add_1, tf.float32)
    add_2 = tf.cast(add_2, tf.float32)

    add_1 = 0.16901935636997223 * (add_1 - 133)
    add_2 = 0.16148914396762848 * add_2

    output_result = tf.add(add_1, add_2)
    output_uint8 = output_result / 0.24699252843856812 + 89
    output_uint8 = tf.math.round(output_uint8)
    output_uint8 = tf.cast(output_uint8, tf.uint8)

    ################## inverted residual 2 expansion ##################
    print('inverted residual 2 expansion')
    new_data = tf.cast(output_uint8, tf.float32)
    new_data -= 89
    s_iw = 0.24699252843856812 * 0.008363455533981323

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_expansion_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_expansion_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 149
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.0020657109562307596
    print(bias)

    output = tf.nn.conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.09814818948507309
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    print()

    ################## inverted residual 2 depthwise ##################
    print('inverted residual 2 depthwise')
    new_data = tf.cast(output_uint8, tf.float32)
    s_iw = 0.09814818948507309 * 0.014716151170432568

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_depthwise_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_depthwise_depthwise_conv_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 120
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.0014443636173382401
    print(bias)

    output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.062810979783535
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    print()

    ################## inverted residual 2 projection ##################
    print('inverted residual 2 projection')
    new_data = tf.cast(output_uint8, tf.float32)
    s_iw = 0.062810979783535 * 0.006514572538435459

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_projection_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_2_projection_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 148
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.00040918667218647897
    print(bias)

    output = tf.nn.conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.0929793044924736 + 138
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    add_2 = tf.identity(output_uint8)
    print()

    ################## inverted residual 3 expansion ##################
    print('inverted residual 3 expansion')
    new_data = tf.cast(output_uint8, tf.float32)
    new_data -= 138
    s_iw = 0.0929793044924736 * 0.005988169927150011

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_expansion_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_expansion_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 137
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.0005567758926190436
    print(bias)

    output = tf.nn.conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.07842949777841568
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    print()

    ################## inverted residual 3 depthwise ##################
    print('inverted residual 3 depthwise')
    new_data = tf.cast(output_uint8, tf.float32)
    s_iw = 0.07842949777841568 * 0.17394107580184937

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_depthwise_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_depthwise_depthwise_conv_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 79
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.013642110861837864
    print(bias)

    output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.05131378769874573
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    print()

    ################## inverted residual 3 projection ##################
    print('inverted residual 3 projection')
    new_data = tf.cast(output_uint8, tf.float32)
    s_iw = 0.05131378769874573 * 0.01676042005419731

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_projection_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_inverted_residual_3_projection_conv_Conv2D_Fold_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 125
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.0008600406581535935
    print(bias)

    output = tf.nn.conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.20826007425785065 + 133
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    add_1 = tf.identity(output_uint8)
    print()

    ################## inverted residual 1 add ##################
    add_1 = tf.cast(add_1, tf.float32)
    add_2 = tf.cast(add_2, tf.float32)

    add_1 = 0.20826007425785065 * (add_1 - 133)
    add_2 = 0.0929793044924736 * (add_2 - 138)

    output_result = tf.add(add_1, add_2)
    output_uint8 = output_result / 0.21021947264671326 + 131
    output_uint8 = tf.math.round(output_uint8)
    output_uint8 = tf.cast(output_uint8, tf.uint8)

    ################## AvgPool ##################
    # method 1
    # new_data = tf.cast(output_uint8, tf.float32)
    # new_data = 0.21021947264671326 * (new_data - 131)
    # output = tf.nn.avg_pool(new_data,
    #                 ksize=[1,25,5,1],
    #                 strides=[1,25,5,1],
    #                 padding='VALID')
    # output = output / 0.21021947264671326 + 131
    # output_uint8 = tf.math.round(output)
    # output_uint8 = tf.cast(output, tf.uint8)

    # method 2 (简化版本，发现scale和zero_point完全可以消除)
    new_data = tf.cast(output_uint8, tf.float32)
    output = tf.nn.avg_pool(new_data,
                    ksize=[1,25,5,1],
                    strides=[1,25,5,1],
                    padding='VALID')
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output, tf.uint8)

    ################## Conv2D ##################
    print('Conv2D')
    new_data = tf.cast(output_uint8, tf.float32)
    new_data -= 131
    s_iw = 0.21021947264671326 * 0.01610618270933628

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_fc_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_fc_conv_Conv2D_bias.npy')
    print(weight.dtype, weight.shape)
    print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32)
    weight -= 143
    weight = tf.transpose(weight, perm=[1,2,3,0])
    print(weight)

    bias = tf.convert_to_tensor(bias, tf.float32)
    bias *= 0.0033858332317322493
    print(bias)

    output = tf.nn.conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,1,1,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None)  # 数据格式，与步长参数配合，决定移动方式
    output = s_iw * output
    output = output + bias
    output = output / 0.1784215271472931 + 129
    
    output = tf.nn.relu(output)
    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)
    add_1 = tf.identity(output_uint8)
    print()

    ################## Reshape ##################
    output_uint8 = tf.squeeze(output_uint8, axis=[1,2])

    ################## Softmax ##################
    new_data = tf.cast(output_uint8, tf.float32)
    new_data = 0.1784215271472931 * (new_data - 129)
    output = tf.nn.softmax(new_data)
    output = output / 0.00390625

    output_uint8 = tf.math.round(output)
    output_uint8 = tf.cast(output_uint8, tf.uint8)

    ################## running ##################

    # print(input_data)
    with tf.Session() as sess:
        output_uint8 = sess.run(output_uint8)
        # output_uint8 = sess.run(add_result)

    return output_uint8

In [210]:
def manual_real(input_data):
    ################## stem conv ##################
    print('stem conv')
    new_data = input_data.reshape(-1, 49, 10, 1)
    new_data = tf.convert_to_tensor(new_data, tf.float32, name='input')
    new_data =  1.1192268133163452 * (new_data - 221.)
    # print(tf.where(new_data < -247.34912109375).eval())
    # print(tf.where(new_data > 38.0537109375).eval())

    weight = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_stem_conv_conv_weights_quant_FakeQuantWithMinMaxVars.npy')
    bias = np.load('test_log/mobilenetv3_quant_eval/weight/MBNetV3-CNN_stem_conv_conv_Conv2D_Fold_bias.npy')
    # print(weight.dtype, weight.shape)
    # print(bias.dtype, bias.shape)

    weight = tf.convert_to_tensor(weight, tf.float32, name='weight')
    weight = 0.0009845112217590213 * (weight - 132.)
    weight = tf.transpose(weight, perm=[1,2,0,3])   # [filter_height, filter_width, in_channels, out_channels]
    low = tf.ones_like(weight) * -0.12866608798503876
    high = tf.ones_like(weight) * 0.12139977514743805
    weight = tf.where(weight < -0.12866608798503876, low, weight)
    weight = tf.where(weight > 0.12139977514743805, high, weight)

    bias = tf.convert_to_tensor(bias, tf.float32, name='bias')
    bias = 0.0011018913937732577 * bias
    # print(weight)
    # print(bias)

    output = tf.nn.depthwise_conv2d(new_data,  # 张量输入
                filter=weight, # 卷积核参数
                strides=[1,2,2,1], # 步长参数
                padding="SAME", # 卷积方式
                data_format=None,  # 数据格式，与步长参数配合，决定移动方式
                name='stem_conv') # 名字，用于tensorboard图形显示时使用

    output_add = tf.add(output, bias, name='add')
    # output_uint8 = tf.cast(output, dtype=tf.uint8)
    low = tf.ones_like(output_add) * 0.16148914396762848
    high = tf.ones_like(output_add) * 41.17973327636719
    output_add = tf.where(output_add < 0.16148914396762848, low, output_add)
    output_add = tf.where(output_add > 41.17973327636719, high, output_add)
    output_add = output_add / 0.16148914396762848
    
    output_relu = tf.nn.relu(output_add)
    output_uint8 = tf.math.round(output_relu, name='round')
    output_uint8 = tf.cast(output_uint8, tf.uint8, name='uint8')
    add_2 = tf.identity(output_uint8)   # 给之后的做加法
    print()

    output_uint8 = sess.run(output_uint8)
    return output_uint8

In [211]:
# test set
set_size = audio_processor.set_size('testing')
tf.logging.info('set_size=%d', set_size)
total_accuracy = 0
for i in range(0, set_size, batch_size):
    test_fingerprints, test_ground_truth = audio_processor.get_data(
        batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
    # print(test_fingerprints.shape)  # (batch_size 490)
    # print(test_ground_truth.shape)  # (batch_size, 12)
    test_fingerprints = fp32_to_uint8(test_fingerprints)
    # output_simulate = manual_int(test_fingerprints)
    output_simulate = manual_real(test_fingerprints)
    output_real = calc(interpreter, test_fingerprints, test_ground_truth)

    # print(output_simulate.shape)
    # print(output_real.shape)
    print(output_simulate)
    print(output_real)
    
    eq = tf.equal(output_real, output_simulate)
    mask = tf.cast(tf.zeros_like(eq), tf.bool)
    neq = tf.reduce_sum(tf.cast(tf.equal(eq, mask), tf.int32))
    print(sess.run(neq), '/', sess.run(tf.size(eq)))

    sys.exit(0)
    # batch_size = min(FLAGS.batch_size, set_size - i)


'''
############################### get all data mean and std_dev ###############################
training_fingerprints, training_ground_truth = audio_processor.get_data(
    -1, 0, model_settings, 0.0, 0.0, 0, 'training', sess)
validation_fingerprints, validation_ground_truth = audio_processor.get_data(
    -1, 0, model_settings, 0.0, 0.0, 0, 'validation', sess)
testing_fingerprints, testing_ground_truth = audio_processor.get_data(
    -1, 0, model_settings, 0.0, 0.0, 0, 'testing', sess)
mean_, std_dev = data_stats(training_fingerprints, validation_fingerprints, testing_fingerprints)
print(mean_, std_dev)
'''

INFO:tensorflow:set_size=4890
stem conv

[[[[ 1  1  1 ... 34  4  1]
   [ 1  6  1 ...  2  1  4]
   [ 1  3  1 ...  4  1  6]
   [ 1  2  1 ...  5  1  6]
   [ 1  3  1 ...  6  1  5]]

  [[ 1  5  5 ... 27 10  1]
   [ 1  5  1 ...  3  1  5]
   [ 1  2  1 ...  3  1  4]
   [ 1  1  1 ...  4  1  4]
   [ 1  2  1 ...  7  1  5]]

  [[ 1  5  1 ... 11  5  1]
   [ 1  6  1 ...  5  1  5]
   [ 1  2  1 ...  4  1  3]
   [ 1  2  1 ...  7  1  4]
   [ 1  1  1 ...  5  1  3]]

  ...

  [[ 1  1  1 ... 14  1  1]
   [ 4  8  1 ...  5  3  1]
   [ 1  4  5 ...  7  2  6]
   [ 1  2  1 ...  3  1  5]
   [ 1  1  1 ...  4  1  5]]

  [[ 1  1  3 ...  8  1  8]
   [ 3  6  1 ... 11  3  2]
   [ 1  4  3 ...  5  1  8]
   [ 1  4  1 ...  3  1  4]
   [ 1  1  1 ...  4  1  6]]

  [[ 1  1  1 ...  1  2  8]
   [ 4  5  1 ... 14  3  8]
   [ 1  3  2 ...  3  1  5]
   [ 1  4  1 ...  4  1  5]
   [ 1  1  1 ...  5  1  5]]]]
[[[[ 0  2  2 ... 34  5  0]
   [ 2  7  1 ...  3  0  5]
   [ 0  4  1 ...  4  0  6]
   [ 0  3  0 ...  6  1  6]
   [ 0  3  1 ...  6  

SystemExit: 0